# Position Reconstruction for XeBRA - Neural Networks

**Status:** August 2019

Position reconstruction for XeBRA using a feedforward neural network with Keras.
Purpose of this notebook: Keras Hyperparameter Tuning using Hyperas.

In [1]:
#############################################
## Use Google Colab:
#############################################
b_GC = False # True or False, also adjust in data()!

if b_GC == True:
    !pip install hyperas
    !pip install hyperopt
    !pip install uproot
    !pip install numpy
    !pip install pandas

In [2]:
#############################################
## Imports:
#############################################

from __future__ import print_function
import numpy as np

from hyperopt import Trials, STATUS_OK, tpe
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils
from keras.optimizers import SGD,Adam

from hyperas import optim
from hyperas.distributions import choice, uniform

import os
import glob
import uproot
import pandas as pd

Using TensorFlow backend.


In [3]:
if b_GC == True:
    ## Import datasets uploaded on Google Drive
    ## by mounting the Google Drive into the runtime.
    from google.colab import drive
    drive.mount('/content/gdrive')
    !ls 'gdrive/My Drive/Colab Notebooks'

In [4]:
#############################################
## Preparing the Data
#############################################

def data():
    """
    Data providing function:

    This function is separated from create_model() so that hyperopt
    won't reload data for each evaluation run.
    """

    ## Selection of file(s) to be used:
    b_GC = False # True or False
    if b_GC == True:
        filename_list = glob.glob("gdrive/My Drive/Simulation Data/optPhot_S2_1.0e+05_pmtGap4.5_RPTFE0.95_LXeAbs100_Tmesh0.89770509_writeEmptyFalse_nop_1.0e+04_*.root")
    else:
        filename_list = glob.glob("/media/ab602/TOSHIBA EXT/Simulation_Data/LCE/optPhot_S2_1.0e+05_pmtGap4.5_RPTFE0.95_LXeAbs100_Tmesh0.89770509_writeEmptyFalse_nop_1.0e+04_*.root")                
    file_split_train = 0.9 # fraction of file used for training
    
    ## Import as pandas dataframes
    branches= ['xp_pri','yp_pri','zp_pri','ntpmthits','pmthits'] # Be careful that the branches are also initilized in _chunk_collect
    import concurrent.futures
    executor = concurrent.futures.ThreadPoolExecutor(8)
    _chunk_collect = pd.DataFrame({'xp_pri' : [],'yp_pri' : [],'zp_pri' : [], 'ntpmthits' : [],'pmthits' : []}) # Initialize empty data frame
    for _chunk in uproot.iterate(filename_list, "events/events",branches, entrysteps=100000, 
                                           outputtype=pd.DataFrame, executor=executor):
        _chunk.columns=branches
        _chunk_collect=_chunk_collect.append(_chunk)

    ## Normalize to hits fractions
    _chunk_collect['pmthits'] = (_chunk_collect['pmthits'] / _chunk_collect['ntpmthits'])

    ## Separate hits for individual PMTs
    ## Takes a while to calculate...
    for i in range(1,8):
        _chunk_collect['pmthits_'+str(i)] = pd.Series([tupl[i] for tupl in np.asarray(_chunk_collect['pmthits'])], index=_chunk_collect.index)

    _chunk_collect = _chunk_collect.drop(columns=['pmthits'])
    _chunk_collect = _chunk_collect.drop(columns=['ntpmthits'])

    ## Separate labels and features
    features=['pmthits_1','pmthits_7']
    labels=['xp_pri','yp_pri']

    ## Split arrays into (random) train and test subsets.
    ## Could also be done with sklearn.model_selection.train_test_split,
    ## but already randomized from simulation (see plots bleow).
    train_label     = _chunk_collect.loc[:int(np.floor(_chunk_collect.shape[0]*file_split_train))-1,labels]
    predict_label   = _chunk_collect.loc[int(np.floor(_chunk_collect.shape[0]*file_split_train)):,labels]
    train_feature   = _chunk_collect.loc[:int(np.floor(_chunk_collect.shape[0]*file_split_train))-1,features[0]:features[1]]
    predict_feature = _chunk_collect.loc[int(np.floor(_chunk_collect.shape[0]*file_split_train)):,features[0]:features[1]]

    del(_chunk_collect)

    ## Convert to numpy arrays
    train_feature   = train_feature.to_numpy()
    train_label     = train_label.to_numpy()
    predict_feature = predict_feature.to_numpy()
    predict_label   = predict_label.to_numpy()

    ## Normalize labels to [-0.5, 0.5] to make network easier to train 
    ## as using smaller, centered values is often better.
    ## Have to rescale later.
    scaling_factor = 70
    train_label = train_label/scaling_factor
    predict_label = predict_label/scaling_factor
    
    ## Return
    return train_feature, train_label, predict_feature, predict_label

In [5]:
#############################################
## Function for optimizing the NN
#############################################

def create_model(train_feature, train_label, predict_feature, predict_label):
    """
    Model providing function:

    Create Keras model with double curly brackets dropped-in as needed.
    Return value has to be a valid python dictionary with two customary keys:
        - loss: Specify a numeric evaluation metric to be minimized
        - status: Just use STATUS_OK and see hyperopt documentation if not feasible
    The last one is optional, though recommended, namely:
        - model: specify the model just created so that we can later use it again.
    """
    
    ## Building the Model
    pmts = int(7) # number of PMTs
    
    model = Sequential()
    model.add(Dense(64, input_shape=(pmts,)))
    model.add(Activation('relu'))
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dense(2))  # x and y
    model.add(Activation('linear'))
    
    ## Compiling the Model
    model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['mean_squared_error']
             )
    
    ## Training the Model
    result = model.fit(train_feature, train_label,
                       epochs={{choice([10, 30, 50, 100])}},
                       batch_size={{choice([8, 16, 32, 64, 128, 256, 512, 1024])}},
                       validation_split={{choice([0.1, 0.2, 0.3, 0.5])}},
                       verbose=2
                      )
    
    ## Get the lowest validation loss of the training epochs
    validation_loss = np.amin(result.history['val_loss']) 
    print('Best validation loss of epoch:', validation_loss)
    return {'loss': validation_loss, 'status': STATUS_OK, 'model': model}

In [ ]:
#############################################
## Optimize the NN
## Make max_evals sufficiently large to cover
## entire paramater space.
#############################################

max_ev = 130

if b_GC == True:
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=max_ev,
                                          trials=Trials(),
                                          eval_space = True,
                                          notebook_name='gdrive/My Drive/Colab Notebooks/XeBRA_Position_Reconstruction_Optimize_NN',
                                          keep_temp=False
                                         )
else: 
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=max_ev,
                                          trials=Trials(),
                                          eval_space = True,
                                          notebook_name='XeBRA_Position_Reconstruction_Optimize_NN',
                                          keep_temp=False
                                         )

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from keras.layers.core import Dense, Dropout, Activation
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    from keras.optimizers import SGD, Adam
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import os
except:
    pass

try:
    import glob
except:
    pass

try:
    import uproot
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    from google.colab import drive
except:
    pass

try:
    import concurrent.futures
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'epochs': hp.choice('epochs', [10, 30, 50, 100]),
        'batch_size': h

/home/ab602/anaconda3/lib/python3.7/site-packages/uproot/tree.py:116: FutureWarning: Int64Index.data is deprecated and will be removed in a future version
  index = awkward.numpy.frombuffer(arrays.index.data, dtype=arrays.index.dtype)


  0%|          | 0/130 [00:00<?, ?it/s, best loss: ?]


W0819 09:37:50.081549 140669794170688 deprecation_wrapper.py:119] From /home/ab602/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.


W0819 09:37:50.092345 140669794170688 deprecation_wrapper.py:119] From /home/ab602/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W0819 09:37:50.093812 140669794170688 deprecation_wrapper.py:119] From /home/ab602/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.


W0819 09:37:50.117677 140669794170688 deprecation_wrapper.py:119] From /home/ab602/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.


W0819 0

Train on 630000 samples, validate on 270000 samples  
Epoch 1/30                                           
 - 145s - loss: 5.2373e-04 - mean_squared_error: 5.2373e-04 - val_loss: 4.4974e-04 - val_mean_squared_error: 4.4974e-04

Epoch 2/30                                           
 - 138s - loss: 4.3526e-04 - mean_squared_error: 4.3526e-04 - val_loss: 4.2527e-04 - val_mean_squared_error: 4.2527e-04

Epoch 3/30                                           
 - 138s - loss: 4.2239e-04 - mean_squared_error: 4.2239e-04 - val_loss: 4.0586e-04 - val_mean_squared_error: 4.0586e-04

Epoch 4/30                                           
 - 135s - loss: 4.1765e-04 - mean_squared_error: 4.1765e-04 - val_loss: 4.1562e-04 - val_mean_squared_error: 4.1562e-04

Epoch 5/30                                           
 - 132s - loss: 4.1422e-04 - mean_squared_error: 4.1422e-04 - val_loss: 4.0909e-04 - val_mean_squared_error: 4.0909e-04

Epoch 6/30                                           
 - 139s - loss: 4

...

Epoch 24/30                                                                                
 - 1s - loss: 3.7515e-04 - mean_squared_error: 3.7515e-04 - val_loss: 4.0772e-04 - val_mean_squared_error: 4.0772e-04

Epoch 25/30                                                                                
 - 1s - loss: 3.7355e-04 - mean_squared_error: 3.7355e-04 - val_loss: 4.6847e-04 - val_mean_squared_error: 4.6847e-04

Epoch 26/30                                                                                
 - 1s - loss: 3.7512e-04 - mean_squared_error: 3.7512e-04 - val_loss: 4.4757e-04 - val_mean_squared_error: 4.4757e-04

Epoch 27/30                                                                                
 - 1s - loss: 3.7396e-04 - mean_squared_error: 3.7396e-04 - val_loss: 4.3197e-04 - val_mean_squared_error: 4.3197e-04

Epoch 28/30                                                                                
 - 1s - loss: 3.7441e-04 - mean_squared_error: 3.7441e-04 - val_

 - 10s - loss: 3.8126e-04 - mean_squared_error: 3.8126e-04 - val_loss: 3.7432e-04 - val_mean_squared_error: 3.7432e-04

Epoch 32/100                                                                              
 - 11s - loss: 3.8104e-04 - mean_squared_error: 3.8104e-04 - val_loss: 3.7059e-04 - val_mean_squared_error: 3.7059e-04

Epoch 33/100                                                                              
 - 11s - loss: 3.8039e-04 - mean_squared_error: 3.8039e-04 - val_loss: 3.7811e-04 - val_mean_squared_error: 3.7811e-04

Epoch 34/100                                                                              
 - 13s - loss: 3.8056e-04 - mean_squared_error: 3.8056e-04 - val_loss: 3.7525e-04 - val_mean_squared_error: 3.7525e-04

Epoch 35/100                                                                              
 - 16s - loss: 3.8059e-04 - mean_squared_error: 3.8059e-04 - val_loss: 3.8038e-04 - val_mean_squared_error: 3.8038e-04

Epoch 36/100                        

Epoch 70/100                                                                              
 - 17s - loss: 3.7682e-04 - mean_squared_error: 3.7682e-04 - val_loss: 3.7676e-04 - val_mean_squared_error: 3.7676e-04

Epoch 71/100                                                                              
 - 20s - loss: 3.7661e-04 - mean_squared_error: 3.7661e-04 - val_loss: 3.8269e-04 - val_mean_squared_error: 3.8269e-04

Epoch 72/100                                                                              
 - 13s - loss: 3.7650e-04 - mean_squared_error: 3.7650e-04 - val_loss: 4.0451e-04 - val_mean_squared_error: 4.0451e-04

Epoch 73/100                                                                              
 - 12s - loss: 3.7643e-04 - mean_squared_error: 3.7643e-04 - val_loss: 3.7517e-04 - val_mean_squared_error: 3.7517e-04

Epoch 74/100                                                                              
 - 16s - loss: 3.7654e-04 - mean_squared_error: 3.7654e-04 - val_

 - 5s - loss: 3.9715e-04 - mean_squared_error: 3.9715e-04 - val_loss: 3.9600e-04 - val_mean_squared_error: 3.9600e-04

Epoch 8/10                                                                                
 - 6s - loss: 3.9432e-04 - mean_squared_error: 3.9432e-04 - val_loss: 3.9749e-04 - val_mean_squared_error: 3.9749e-04

Epoch 9/10                                                                                
 - 7s - loss: 3.9342e-04 - mean_squared_error: 3.9342e-04 - val_loss: 4.0273e-04 - val_mean_squared_error: 4.0273e-04

Epoch 10/10                                                                               
 - 6s - loss: 3.9041e-04 - mean_squared_error: 3.9041e-04 - val_loss: 3.8678e-04 - val_mean_squared_error: 3.8678e-04

Best validation loss of epoch:                                                            
0.0003867760162914379                                                                     
Train on 810000 samples, validate on 90000 samples                   

Epoch 35/100                                                                              
 - 114s - loss: 3.9284e-04 - mean_squared_error: 3.9284e-04 - val_loss: 3.9815e-04 - val_mean_squared_error: 3.9815e-04

Epoch 36/100                                                                              
 - 107s - loss: 3.9247e-04 - mean_squared_error: 3.9247e-04 - val_loss: 4.4964e-04 - val_mean_squared_error: 4.4964e-04

Epoch 37/100                                                                              
 - 122s - loss: 3.9250e-04 - mean_squared_error: 3.9250e-04 - val_loss: 3.9724e-04 - val_mean_squared_error: 3.9724e-04

Epoch 38/100                                                                              
 - 108s - loss: 3.9236e-04 - mean_squared_error: 3.9236e-04 - val_loss: 3.8297e-04 - val_mean_squared_error: 3.8297e-04

Epoch 39/100                                                                              
 - 100s - loss: 3.9262e-04 - mean_squared_error: 3.9262e-04 -

In [ ]:
X_train, Y_train, X_test, Y_test = data()
print("Evalutation of best performing model:")
print(best_model.evaluate(X_test, Y_test))
print("Best performing model chosen hyper-parameters:")
print(best_run)

best_model.save('best_model.h5')

In [ ]:
## Save optimization results to txt file

fh = open("Optimization_results.txt", "w") 

fh.write("Evalutation of best performing model:\n")
fh.write(str(best_model.evaluate(X_test, Y_test))+"\n")
fh.write("Best performing model chosen hyper-parameters:\n")
fh.write(str(best_run)+"\n")
 
fh.close() 